# 멜론 크롤링
1.노래 리스트 받아와서 쿼리 바꿔서 검색
: 노래 리스트는 json파일로 저장 - key=노래, value:가수이름
그 안에서 id 찾아서 -> 노래 id 리스트업
2.노래 id로 노래상세페이지 쿼리바꿔가면서 데이터 크롤링 - 발매일, 장르, 좋아요수, 댓글수

In [4]:
import time
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests
import re
import json

In [13]:
gaon_df = pd.read_csv('./gaon_data_sp.csv')

music_list = gaon_df['music'].unique().tolist()
print(len(music_list), len(gaon_df))

118 1000


In [7]:
# 뮤비가 없으면 모든 데이터가 저장이 되지 않는 오류 해결

gaon_df = pd.read_csv('./gaon_data_sp.csv')
gaon_df = gaon_df[:10]
music_list = gaon_df['music'].unique().tolist()
base_url = "https://www.melon.com/"
id_url = 'search/total/index.htm?q='


for music in music_list:
    
    singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
    
    # 이름에 괄호가 있는 경우만 변수에 괄호 안의 내용 저장 후 괄호 삭제
    if singer.find('(') != -1:
        p = re.compile('\(([^)]+)')
        singer_bracket = p.findall(singer)[0]
        singer = re.sub(' \([^)]*\)','', singer) #정규표현식으로 괄호 안의 내용 모두 삭제
    else:
        singer_bracket = singer

    # 가수 이름의 공백을 '+'로 바꿔줌
    singer = singer.replace(' ', '+')

    music = re.sub(' \([^)]*\)', '', music) #정규표현식으로 괄호 안의 내용 모두 삭제
    music = music.replace(' ', '+')

    # 오류 방지를 위해 가수 3명이상 같이 부른 노래면 맨 앞 사람만으로 검색하기
    singer_list = singer.split('+')

    print(f'음악: {music}, 가수:{singer}, 가수괄호:{singer_bracket}, singer_list:{singer_list}')

    try:
        try:
            for singer in singer_list:
                URL = base_url + id_url + singer + '+' + music
                print(URL)
                header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
                response = requests.get(URL,headers=header)
                soup = BeautifulSoup(response.text, 'html.parser')

                links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
                music_id = links.attrs['value']
                print('music_id: ', music_id)

                time.sleep(2)

                # 노래 데이터 추출
                URL_1 = base_url + f'song/detail.htm?songId={music_id}'
                response_1 = requests.get(URL_1,headers=header)
                soup = BeautifulSoup(response_1.text, 'html.parser')

                # 발매일, 장르
                release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
                category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
                
                time.sleep(2)

                # 좋아요 가져오기위해 json으로 접근
                URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={music_id}'
                response_2 = requests.get(URL_2,headers=header).text

                like_json = json.loads(response_2)
                music_like = like_json['contsLike'][0]['SUMMCNT']


                # 뮤비 고유 code 가져오기 - 상세페이지에 뮤비 없는 경우 pass
                try:
                    movie = soup.select_one('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li > div.thumb > a')
                    movie = movie.attrs['href']
                    movie = re.sub('[a-zA-z:.(\')]', '', movie)
                    movie_id = movie.split(', ')[1]
                    time.sleep(2)

                    # 뮤비 고유 코드 이용해서 json 접근해서 뮤비 뷰수와 좋아요 가져오기
                    URL_3 = f'https://www.melon.com/commonlike/getMvLikeWithReadCnt.json?contsIds={movie_id}'
                    response_3 = requests.get(URL_3, headers=header).text
                    movie_json = json.loads(response_3)
                    movie_like = movie_json['contsLike'][0]['LOVECNT']
                    movie_views = movie_json['contsLike'][0]['READCNT']
                    
                    print(f'뮤비 있음 노래제목:{music} 뮤비좋아요:{movie_like} 뮤비뷰수:{movie_views}')
    
                except:
                    print(f'뮤비 없음 노래제목:{music}')

                break
 
        # 가수 이름으로 검색해서 없으면 가수이름 괄호 안의 이름으로 검색 singer_bracket
        except:
            URL = base_url + id_url + singer_bracket + '+' + music
            print(URL)
            header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
            response = requests.get(URL,headers=header)
            soup = BeautifulSoup(response.text, 'html.parser')

            time.sleep(2)

            links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
            music_id = links.attrs['value']
            print('music_id: ', music_id)
            
            # 노래 데이터 추출
            URL_1 = base_url + f'song/detail.htm?songId={music_id}'
            response_1 = requests.get(URL_1,headers=header)
            soup = BeautifulSoup(response_1.text, 'html.parser')

            # 발매일, 장르
            release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
            category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
            
            time.sleep(2)

            # 좋아요 가져오기위해 json으로 접근
            URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={music_id}'
            response_2 = requests.get(URL_2,headers=header).text

            like_json = json.loads(response_2)
            music_like = like_json['contsLike'][0]['SUMMCNT']


            # 뮤비 고유 code 가져오기 - 상세페이지에 뮤비 없는 경우 pass
            try:
                time.sleep(2)
                movie = soup.select_one('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li > div.thumb > a')
                movie = movie.attrs['href']
                movie = re.sub('[a-zA-z:.(\')]', '', movie)
                movie_id = movie.split(', ')[1]

                time.sleep(2)
                # 뮤비 고유 코드 이용해서 json 접근해서 뮤비 뷰수와 좋아요 가져오기
                URL_3 = f'https://www.melon.com/commonlike/getMvLikeWithReadCnt.json?contsIds={movie_id}'
                response_3 = requests.get(URL_3, headers=header).text
                movie_json = json.loads(response_3)
                movie_like = movie_json['contsLike'][0]['LOVECNT']
                movie_views = movie_json['contsLike'][0]['READCNT']

                print(f'뮤비 있음 노래제목:{music} 뮤비좋아요:{movie_like} 뮤비뷰수:{movie_views}')
            except:
                print(f'뮤비 없음 노래제목:{music}')
                
    except:
        print('오류 발생 : 해당 노래 검색값 없음')
        pass

음악: Dancing+Queen, 가수:소녀시대, 가수괄호:소녀시대, singer_list:['소녀시대']
https://www.melon.com/search/total/index.htm?q=소녀시대+Dancing+Queen
music_id:  3994911
뮤비 있음 노래제목:Dancing+Queen 뮤비좋아요:1495 뮤비뷰수:627890
음악: 사랑은+이렇게, 가수:케이윌, 가수괄호:K.Will, singer_list:['케이윌']
https://www.melon.com/search/total/index.htm?q=케이윌+사랑은+이렇게
music_id:  3995922
뮤비 있음 노래제목:사랑은+이렇게 뮤비좋아요:381 뮤비뷰수:48870
음악: 되돌리다, 가수:이승기, 가수괄호:이승기, singer_list:['이승기']
https://www.melon.com/search/total/index.htm?q=이승기+되돌리다
music_id:  3968836
뮤비 있음 노래제목:되돌리다 뮤비좋아요:963 뮤비뷰수:140511
음악: 카페인, 가수:양요섭, 가수괄호:양요섭, singer_list:['양요섭']
https://www.melon.com/search/total/index.htm?q=양요섭+카페인
music_id:  3971021
뮤비 있음 노래제목:카페인 뮤비좋아요:3875 뮤비뷰수:228636
음악: 매력있어, 가수:AKMU, 가수괄호:악뮤, singer_list:['AKMU']
https://www.melon.com/search/total/index.htm?q=AKMU+매력있어
music_id:  3987723
뮤비 없음 노래제목:매력있어
음악: 크리스마스니까, 가수:성시경,+박효신,+이석훈,+서인국,+빅스, 가수괄호:VIXX, singer_list:['성시경,', '박효신,', '이석훈,', '서인국,', '빅스']
https://www.melon.com/search/total/index.htm?q=성시경,+크리스마스니까
music_id:  3

In [ ]:
# 2013년 1주차 장현승,정은지,김남주 노래 일년전에 오류 뜨는 이유
# 오류 : 검색해도 노래가 아니고 mv만 떠서 값 못 찾아서 오류
# 오류 이유 : 장현승이 최근에 이름을 able로 바꿔서 able으로 검색해야 노래가 나와서......


In [11]:
# 0. 최종df에서 노래랑 제목 가져와서 검색
gaon_df = pd.read_csv('./gaon_data_sp.csv')
music_list = gaon_df['music'].unique().tolist()
base_url = "https://www.melon.com/"

for music in music_list:
    
    singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
    
    # 이름에 괄호가 있는 경우만 변수에 괄호 안의 내용 저장 후 괄호 삭제
    if singer.find('(') != -1:
        p = re.compile('\(([^)]+)')
        singer_bracket = p.findall(singer)[0]
        singer = re.sub(' \([^)]*\)','', singer) #정규표현식으로 괄호 안의 내용 모두 삭제
    else:
        singer_bracket = singer

    # 가수 이름의 공백을 '+'로 바꿔줌
    singer = singer.replace(' ', '+')

    music = re.sub(' \([^)]*\)', '', music) #정규표현식으로 괄호 안의 내용 모두 삭제
    music = music.replace(' ', '+')

    # 오류 방지를 위해 가수 3명이상 같이 부른 노래면 맨 앞 사람만으로 검색하기
    singer_list = singer.split('+')

    # if len(singer_list) > 2:
    #     singer_search = singer_list[0]
    # else:
    #     singer_search = singer

    id_url = 'search/total/index.htm?q='
    time.sleep(3)
    # singer을 바꿔서 검색
    # 가수가 3명 이상이면 singer_list로 가수명 쪼개고 그 안에서 for문으로 오류 안나고 값 찾을때까지 반복
    # 만약 위에 거 안된다면 괄호안의 내용으로 검색 
    try:
        try:
            for singer in singer_list:
                URL = base_url + id_url + singer + '+' + music
                header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
                response = requests.get(URL,headers=header)
                soup = BeautifulSoup(response.text, 'html.parser')

                links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
                music_id = links.attrs['value']
                print(f'singef: {singer}, music_id: {music_id}')
                break
        except:
            URL = base_url + id_url + singer_bracket + '+' + music
            header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
            response = requests.get(URL,headers=header)
            soup = BeautifulSoup(response.text, 'html.parser')

            links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
            music_id = links.attrs['value']
            print(f'singef: {singer}, music_id: {music_id}')
    except:
        print('오류 발생 : 값 없음')
        pass
    time.sleep(3)


오류 발생 : 값 없음
오류 발생 : 값 없음


KeyboardInterrupt: 

In [ ]:
# 처리해야하는거
# singer_list 가수 여러명이면 콤마로 이어져있어서
# 다 빼주는게 좋을 거 같긴 함
# ex) 장현승, 정은지, 김남주
# singer_list[0] 이 '장현승,'

In [109]:
# 백업
# 0. 최종df에서 노래랑 제목 가져와서 검색
gaon_df = pd.read_csv('./gaon_data_sp.csv')
music_list = gaon_df['music'].unique().tolist()
base_url = "https://www.melon.com/"

for music in music_list:
    
    singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
    
    # 이름에 괄호가 있는 경우만 변수에 괄호 안의 내용 저장 후 괄호 삭제
    if singer.find('(') != -1:
        p = re.compile('\(([^)]+)')
        singer_bracket = p.findall(singer)[0]
        singer = re.sub(' \([^)]*\)','', singer) #정규표현식으로 괄호 안의 내용 모두 삭제
    else:
        singer_bracket = singer

    # 가수 이름의 공백을 '+'로 바꿔줌
    singer = singer.replace(' ', '+')

    music = re.sub(' \([^)]*\)', '', music) #정규표현식으로 괄호 안의 내용 모두 삭제
    music = music.replace(' ', '+')

    # 오류 방지를 위해 가수 3명이상 같이 부른 노래면 맨 앞 사람만으로 검색하기
    singer_list = singer.split('+')
    print('singer_list', singer_list)
    if len(singer_list) > 2:
        singer_search = singer_list[0]
        print('가수 3명이상', singer_search)
    else:
        singer_search = singer
        print('가수', singer)

    id_url = 'search/total/index.htm?q='

    # 검색 결과가 없는 경우, 가수 이름 괄호 안의 내용으로 검색
    try:
        URL = base_url + id_url + singer_search + '+' + music
        print(URL)
        header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
        response = requests.get(URL,headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')

        links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
        music_id = links.attrs['value']
    except:

        URL = base_url + id_url + singer_bracket + '+' + music
        print(URL)
        header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
        response = requests.get(URL,headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')

        links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
        music_id = links.attrs['value']

singer_list ['소녀시대']
가수 소녀시대
https://www.melon.com/search/total/index.htm?q=소녀시대+Dancing+Queen
singer_list ['케이윌']
가수 케이윌
https://www.melon.com/search/total/index.htm?q=케이윌+사랑은+이렇게
singer_list ['이승기']
가수 이승기
https://www.melon.com/search/total/index.htm?q=이승기+되돌리다
singer_list ['양요섭']
가수 양요섭
https://www.melon.com/search/total/index.htm?q=양요섭+카페인
singer_list ['AKMU']
가수 AKMU
https://www.melon.com/search/total/index.htm?q=AKMU+매력있어
singer_list ['성시경,', '박효신,', '이석훈,', '서인국,', '빅스']
가수 3명이상 성시경,
https://www.melon.com/search/total/index.htm?q=성시경,+크리스마스니까
singer_list ['써니힐']
가수 써니힐
https://www.melon.com/search/total/index.htm?q=써니힐+Goodbye+To+Romance
singer_list ['소녀시대']
가수 소녀시대
https://www.melon.com/search/total/index.htm?q=소녀시대+I+Got+A+Boy
singer_list ['효린,', '전효성,', '현아,', '니콜,', '나나']
가수 3명이상 효린,
https://www.melon.com/search/total/index.htm?q=효린,+이사람
singer_list ['정진운,', '엘조,', '이준,', '호야,', '이기광']
가수 3명이상 정진운,
https://www.melon.com/search/total/index.htm?q=정진운,+Yesterday
singer_list ['강

AttributeError: 'NoneType' object has no attribute 'attrs'

In [54]:
gaon_df['singer'].tolist()

['소녀시대',
 '케이윌 (K.Will)',
 '이승기',
 '양요섭',
 'AKMU (악뮤)',
 '성시경, 박효신, 이석훈, 서인국, 빅스 (VIXX)',
 '써니힐 (Sunny Hill)',
 '소녀시대',
 '양요섭',
 '효린, 전효성, 현아, 니콜, 나나',
 '정진운 (2am), 엘조 (틴탑), 이준 (엠블랙), 호야 (인피니트), 이기광 (비스트)',
 '소녀시대',
 '강지영, 보라, 한선화, 허가윤, 리지',
 '백지영',
 'AKMU (악뮤)',
 '제아',
 '더 원 (The One)',
 '이승기',
 '장현승, 정은지, 김남주',
 '써니힐 (Sunny Hill)',
 '소녀시대',
 '정형돈',
 '길 (리쌍)',
 '김재중 (JYJ)',
 '소녀시대',
 '써니힐 (Sunny Hill)',
 '제아',
 '백지영',
 '효린, 전효성, 현아, 니콜, 나나',
 '노홍철',
 '유재석',
 '이승기',
 '장현승, 정은지, 김남주',
 '버벌진트 (Verbla Jint)',
 '양요섭',
 'AKMU (악뮤)',
 '배치기',
 '백지영',
 '이승기',
 '긱스 (Geeks), 소유',
 '씨엔블루 (CNBLUE)',
 '버벌진트 (Verbla Jint)',
 '양요섭',
 'AKMU (악뮤)',
 '정형돈',
 '김재중 (JYJ)',
 'B.A.P',
 '인피니트H',
 '소녀시대',
 '김보경',
 '소녀시대',
 '제아',
 '인피니트H',
 '써니힐 (Sunny Hill)',
 '효린, 전효성, 현아, 니콜, 나나',
 '노을',
 '리쌍',
 '배치기',
 '알리 (ALi)',
 '포미닛 투윤 (2Yoon)',
 '이정',
 '허각, 유승우',
 '씨엔블루 (CNBLUE)',
 '정형돈',
 '써니힐 (Sunny Hill)',
 '디셈버 (December)',
 '씨스타19',
 '에일리 (Ailee)',
 '백지영',
 '씨스타19',
 '긱스 (Geeks), 소유',
 '배치기',
 '리쌍',
 '김보경',
 'MFB

In [4]:
# 1. 노래 리스트에서 가져와서 노래별로 id 찾기
# music_list = ['and july', 'cupid', '꽃']
# singer_list = ['heize', '피프티피프티', '지수']

# 띄어쓰기는 +로 바꿔주기
music_list = [m.replace(' ', '+') for m in music_list]
singer_list = [s.replace(' ', '+') for s in singer_list]

In [52]:
singer_list

NameError: name 'singer_list' is not defined

In [45]:
music_list

['Dancing+Queen',
 '사랑은+이렇게',
 '되돌리다',
 '카페인',
 '매력있어',
 '크리스마스니까',
 'Goodbye+To+Romance',
 'I+Got+A+Boy',
 '이사람',
 'Yesterday',
 '인어공주',
 '싫다',
 '안아보자',
 '그+사람+(원곡가수+이승철)',
 '일년전에',
 'Express+999',
 '강북멋쟁이',
 '엄마를+닮았네',
 'One+Kiss',
 '노가르시아',
 '메뚜기월드',
 '시작이+좋아+(Feat.+강민희)',
 '눈물샤워+(Feat.+에일리)',
 'Officially+Missing+You,+Too',
 'I’m+Sorry',
 'Mine',
 '빗소리',
 '니가+없을+때+(Feat.+Zion.T)',
 '혼자라고+생각말기',
 'Special+Girl+(Feat.+Bumkey)',
 '하지+못한+말',
 '눈물+(Feat.+유진)',
 '지우개',
 '24/7',
 '너였으면+좋겠어',
 '모노드라마',
 '기억을+걷다보면',
 '있다+없으니까+(Gone+Not+Around+Any+Longer)',
 '얼음꽃',
 '나도+여자인데+(A+Girl+In+Love)',
 'Sweet+Dream',
 '저녁+하늘',
 '이별재회',
 '행복하니',
 '섹시+보이+(Feat.+영지)',
 '너+땜에+못살아+(Feat.+버벌진트)',
 '그런+너+(Disturbance)',
 '먹지',
 'Call+Me+Maybe',
 '안녕+나야',
 'One+Shot',
 '러브+레시피',
 'Dream+Girl',
 '라면인건가',
 '어이없네',
 '어떤가요',
 '아름다워+(Beautiful)',
 '1440',
 '돌스+(Dolls)',
 '겨울사랑',
 '눈꽃',
 '이게+사랑이+아니면+(Feat.+에일리)',
 '모르시나요',
 '선인장+(Duet)',
 '긴+생머리+그녀+(Miss+Right)',
 'Black+Paradise',
 '귀요미송',
 '될+대로+되라고+해+(느낌+So+Go

In [46]:

# 2. 노래 id 찾아서 code_list 만들기
# https://www.melon.com/search/total/index.htm?q=heize+and+july

code_list=[]
for idx, music in enumerate(music_list):
    base_url = 'https://www.melon.com/search/total/index.htm?q='
    URL = base_url + singer_list[idx] + '+' + music
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    response = requests.get(URL,headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
    code = links.attrs['value']
    code_list.append(code)
code_list

NameError: name 'singer_list' is not defined

In [51]:
music = music_list[1]
singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
singer = singer.replace(' ', '+')
singer

'케이윌+(K.Will)'

In [48]:
gaon_df = pd.read_csv('./gaon_data_sp.csv')
music_list = gaon_df['music'].unique().tolist()
base_url = "https://www.melon.com/"

for music in music_list:
    # singer찾아서 meta로 담아서 보내기
    singer = gaon_df['singer'].loc[gaon_df['music']==music].unique()[0]
    singer = singer.replace(' ', '+')
    music = music.replace(' ', '+')
    
    id_url = 'search/total/index.htm?q='

    URL = base_url + id_url + singer + '+' + music
    print(URL)
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    response = requests.get(URL,headers=header)
    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.select_one('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(1) > div > input')
    music_id = links.attrs['value']
    print(music_id)

https://www.melon.com/search/total/index.htm?q=소녀시대+Dancing+Queen
3994911
https://www.melon.com/search/total/index.htm?q=케이윌+(K.Will)+사랑은+이렇게


AttributeError: 'NoneType' object has no attribute 'attrs'

In [ ]:
# 노래 제목, 가수 이름으로 url에 넣어서 노래의 고유 아이디 구하기
# 노래 id로 발매일, 장르, 좋아요수, 댓글수, 뮤비고유코드 얻기
    # 뮤비 고유 코드로는 뮤비 좋아요수와 뷰 수 얻기


In [145]:
# 노래 id로 노래상세페이지 쿼리바꿔가면서 데이터 크롤링 
# - 발매일, 장르, 좋아요수, 댓글수, 뮤비고유코드 ( -> 뮤비 좋아요, 뷰)

import json
import re

for code in code_list:
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36"}
    URL = f'https://www.melon.com/song/detail.htm?songId={code}'
    response_1 = requests.get(URL,headers=header)
    soup = BeautifulSoup(response_1.text, 'html.parser')

    # 발매일, 장르
    release_date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)').text
    category = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
    
    
    # 좋아요 가져오기위해 json으로 접근
    URL_2 = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={code}'
    response_2 = requests.get(URL_2,headers=header).text

    like_json = json.loads(response_2)
    like_count = like_json['contsLike'][0]['SUMMCNT']

    print(f'노래코드:{code} 발매일{release_date}, 장르{category} 좋아요{like_count}')
    print('--------------------------------------------------------------')


    # 뮤비 고유 code 가져오기
    movie = soup.select_one('#conts > div.section_movie > div.service_list_video.type03.d_video_list > ul > li > div.thumb > a')
    movie = movie.attrs['href']
    movie = re.sub('[a-zA-z:.(\')]', '', movie)
    movie_code = movie.split(', ')[1]

    # 뮤비 고유 코드 이용해서 json 접근해서 뮤비 뷰수와 좋아요 가져오기
    URL_3 = f'https://www.melon.com/commonlike/getMvLikeWithReadCnt.json?contsIds={movie_code}'
    response_3 = requests.get(URL_3, headers=header).text
    movie_json = json.loads(response_3)
    movie_like = movie_json['contsLike'][0]['LOVECNT']
    movie_views = movie_json['contsLike'][0]['READCNT']
    print(f'뮤비 코드:{movie_code} 뮤비좋아요:{movie_like}, 뮤비뷰수{movie_views}')


뮤비 코드:50152059 뮤비좋아요:2309, 뮤비뷰수588781
노래코드:8271355 발매일2016.07.18, 장르랩/힙합 좋아요182074
--------------------------------------------------------------
뮤비 코드:50260014 뮤비좋아요:217, 뮤비뷰수8829
노래코드:36206259 발매일2023.02.24, 장르댄스 좋아요50755
--------------------------------------------------------------
뮤비 코드:50260334 뮤비좋아요:1488, 뮤비뷰수116284
노래코드:36331121 발매일2023.03.31, 장르댄스 좋아요86179
--------------------------------------------------------------
